In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
import os
import glob
import guitarpro
import librosa, soundfile
import numpy as np
import scipy
from utils import *
from operations import *

## Break multi-track gtp files into single-track gtp files

In [ ]:
MULTI_TRACK_DIR = "/Volumes/MacOnly/UG_raw/all_time_top_by_hits"
SINGLE_TRACK_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/single_track_gtps"

i = 0
for file in glob.glob(os.path.join(MULTI_TRACK_DIR, "*.gp*")):
    i += 1
    print(f"processing file {i}: {file.split('/')[-1]}")
    get_single_tracks(
        file=file,
        output_dir=SINGLE_TRACK_DIR,
        unify_volume=True,
        force_clean=False,
        disable_repeats=False,
        disable_mixTableChange=False,
    )



In [ ]:
MULTI_TRACK_DIR = "/Volumes/MacOnly/UG_raw/all_time_top_by_hits"
# clean_single_track_gtps contains gtp files that have:
# unified volume, clean guitar tone, no repeats, no mixTableChanges
CLEAN_SINGLE_TRACK_DIR = (
    "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/clean_single_track_gtps"
)

i = 0
for file in glob.glob(os.path.join(MULTI_TRACK_DIR, "*.gp*")):
    i += 1
    print(f"processing file {i}: {file.split('/')[-1]}")
    get_single_tracks(file=file, output_dir=CLEAN_SINGLE_TRACK_DIR)



## Segment single-track audio into poly and mono segments (GT)

In [4]:
# a test
file_name = "DragonForce - Through The Fire And Flames_Lead 2.gp5"
song = guitarpro.parse(os.path.join(CLEAN_SINGLE_TRACK_DIR, file_name))
poly, mono = poly_vs_mono(song)


In [5]:
poly


[[37.2, 38.400000000000006],
 [288.6, 289.2],
 [289.8, 290.4],
 [414.0, 418.79999999999995]]

In [6]:
mono


[[0.0, 37.2],
 [38.4, 288.59999999999997],
 [289.2, 289.8],
 [290.4, 414.0],
 [418.8, 433.2]]

In [ ]:
CLEAN_SINGLE_TRACK_DIR = (
    "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/clean_single_track_gtps"
)
SINGLE_TRACK_AUDIO_DIR = (
    "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/clean_single_track_audio"
)
POLY_SEGMENTS_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/poly_audio_segments"
MONO_SEGMENTS_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/mono_audio_segments"

i = 0
for file in glob.glob(os.path.join(CLEAN_SINGLE_TRACK_DIR, "*.gp5")):
    i += 1
    track_title, _ = os.path.splitext(file.split("/")[-1])
    print(f"processing file {i}: {track_title}")

    song = guitarpro.parse(file)
    poly, mono = poly_vs_mono(song)

    y, sr = librosa.load(
        os.path.join(SINGLE_TRACK_AUDIO_DIR, track_title + ".wav"), sr=None, mono=True
    )

    for j, poly_segment in enumerate(poly):
        start_sec = poly_segment[0]
        end_sec = poly_segment[1]
        segment = y[int(start_sec * sr) : int(end_sec * sr)]
        # write segment to file, store it in the "poly" folder
        segment_file_name = "{}_{}_{}s.wav".format(
            track_title, j, int(start_sec)
        )
        soundfile.write(os.path.join(POLY_SEGMENTS_DIR, segment_file_name), segment, sr)

    for j, mono_segment in enumerate(mono):
        start_sec = mono_segment[0]
        end_sec = mono_segment[1]
        segment = y[int(start_sec * sr) : int(end_sec * sr)]
        # write segment to file, store it in the "mono" folder
        segment_file_name = "{}_{}_{}s.wav".format(track_title, j, int(start_sec))
        soundfile.write(os.path.join(MONO_SEGMENTS_DIR, segment_file_name), segment, sr)



## Generate frames (features and labels) from the poly and mono segments

In [ ]:
# frame-level features: mel-spectrum (compute via mel-spectrogram)
FRAME_SIZE = 2048
HOP_SIZE = 2048
MONO_LABEL = 0

MONO_SEGMENTS_DIR = "/content/mono_audio_segments"

i = 0
features_with_label = []

for file in glob.glob(os.path.join(MONO_SEGMENTS_DIR, "*.wav")):
    i += 1
    title, _ = os.path.splitext(file.split("/")[-1])
    print(f"processing file {i}: {title}")

    y, sr = librosa.load(file, sr=None, mono=True)
    mel_spec = librosa.feature.melspectrogram(y, sr, n_fft=FRAME_SIZE, hop_length=HOP_SIZE, center=False)

    no_frames = mel_spec.shape[1]
    columns_to_delete = []

    for j in range(no_frames):
        if np.all(mel_spec[:, j] == 0):
            columns_to_delete.append(j)
    mel_spec = np.delete(mel_spec, columns_to_delete, axis=1)
    print(f"deleted {len(columns_to_delete)} empty frames")
    # zero for mono
    label = np.zeros((1, mel_spec.shape[1]))
    feature_with_label = np.append(mel_spec, label, axis=0)
    features_with_label.append(feature_with_label)

mono_data = np.concatenate(features_with_label, axis=1)
mono_data.shape # (129, 626587)

In [43]:
np.savez_compressed("/content/drive/MyDrive/THESIS CODE/mono.npz", mono_data)   # ~300M
# np.save("/content/drive/MyDrive/THESIS CODE/mono.npy", mono_data)   # ~600M

In [ ]:
# frame-level features: mel-spectrum (compute via mel-spectrogram)
FRAME_SIZE = 2048
HOP_SIZE = 2048
POLY_LABEL = 1

POLY_SEGMENTS_DIR = "/content/poly_audio_segments"

i = 0
features_with_label = []

for file in glob.glob(os.path.join(POLY_SEGMENTS_DIR, "*.wav")):
    i += 1
    title, _ = os.path.splitext(file.split("/")[-1])
    print(f"processing file {i}: {title}")

    y, sr = librosa.load(file, sr=None, mono=True)
    mel_spec = librosa.feature.melspectrogram(y, sr, n_fft=FRAME_SIZE, hop_length=HOP_SIZE, center=False)

    no_frames = mel_spec.shape[1]
    columns_to_delete = []

    for j in range(no_frames):
        if np.all(mel_spec[:, j] == 0):
            columns_to_delete.append(j)
    mel_spec = np.delete(mel_spec, columns_to_delete, axis=1)
    print(f"deleted {len(columns_to_delete)} empty frames")
    # one for poly
    label = np.ones((1, mel_spec.shape[1]))
    feature_with_label = np.append(mel_spec, label, axis=0)
    features_with_label.append(feature_with_label)

poly_data = np.concatenate(features_with_label, axis=1)
poly_data.shape # (129, 551288)

In [ ]:
np.savez_compressed("/content/drive/MyDrive/THESIS CODE/poly.npz", poly_data)

## Train sklearn classifier

In [4]:
mono = np.load("/Volumes/MacOnly/UG_proc/all_time_top_by_hits/mono.npz")
poly = np.load("/Volumes/MacOnly/UG_proc/all_time_top_by_hits/poly.npz")
print(mono.files)
print(poly.files)

['arr_0']
['arr_0']


In [5]:
with np.load("/Volumes/MacOnly/UG_proc/all_time_top_by_hits/mono.npz") as data:
    mono = data["arr_0"]
print(mono.shape)
with np.load("/Volumes/MacOnly/UG_proc/all_time_top_by_hits/poly.npz") as data:
    poly = data["arr_0"]
print(poly.shape)
data = np.concatenate((mono, poly), axis=1)
print(data.shape)

(129, 626587)
(129, 551288)
(129, 1177875)


In [6]:
X = data[:-1, :]
X = X.transpose()   # transpose so that X and y have the same size on the first dimension 
print(X.shape)
y = data[-1, :]
print(y.shape)

(1177875, 128)
(1177875,)


In [7]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.utils import shuffle

In [ ]:
X_shuffled, y_shuffled = shuffle(X, y)
lr = LogisticRegression()
cross_val_score(lr, X_shuffled, y_shuffled, cv=3)
# [0.68747023, 0.68724101, 0.68916905]

In [ ]:
X_shuffled, y_shuffled = shuffle(X, y)
clf = SVC()
cross_val_score(clf, X_shuffled[:400000, :], y_shuffled[:400000,], cv=3)
# [0.85315734, 0.85660717, 0.85673567]

In [ ]:
# long-running SVM training on full data
X, y = shuffle(X, y)

X_train, X_test, y_train, y_test = train_test_split(X[:400000, :], y[:400000,], test_size=0.2)
svm = SVC()
svm = svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

## Break the single-track gtp files into 4-bar phrases (necessary??)

In [ ]:
SINGLE_TRACK_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/single_track_gtps"
PHRASE_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/phrase_gtps"
for file in glob.glob(os.path.join(SINGLE_TRACK_DIR, "*.gp*")):
    print(f"processing {file.split('/')[-1]}")
    get_phrases(file, PHRASE_DIR)



In [12]:
PHRASE_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/phrase_gtps"
file_name = "ACDC - Highway To Hell (ver 3)_Guitar 2_6.gp5"
song = guitarpro.parse(os.path.join(PHRASE_DIR, file_name))
poly, mono = poly_vs_mono(song)


BELOW IS SUBJECT TO CHANGE

In [ ]:
JSON_DIR = "/content/drive/MyDrive/UG/proc_phrases_anno"
for file in glob.glob(os.path.join(PHRASE_DIR, "*.gp*")):
    print(f"processing {file.split('/')[-1]}")
    # get_anno(file, JSON_DIR)



In [ ]:
# "Led Zeppelin - Stairway To Heaven_4_34" seems to be empty because of tied notes from the previous phrase


In [29]:
file = "/content/drive/MyDrive/tempochange.gp5"
song = guitarpro.parse(file)
song.tracks[0].measures[1].voices[0].beats[2].effect.mixTableChange.tempo


MixTableItem(value=200, duration=0, allTracks=False)

In [33]:
song.tracks[0].measures[1].voices[0].beats[0].start


4800

In [41]:
song.tracks[0].measures[1].voices[0].beats[1].duration.time


960

In [53]:
song.tracks[0].measures[1].voices[0].beats


In [37]:
song.tracks[0].measures[1].voices[0].beats[3].start


7680

In [58]:
file = "/content/drive/MyDrive/repeat.gp5"
song = guitarpro.parse(file)
song.tracks[0].measures[0].voices[0].beats[-1].duration.time


1920